# 入门训练：RHF 与 RMP2 能量

在这一节中，我们会使用 PySCF 的库进行 RHF 与 RMP2 能量的计算，同时对其中的部分中间矩阵进行输出与性质的考察．

## 修改库函数的准备

PySCF 绝大部分与量子化学方法本身有关的内容使用 Python 编程，因此我们可以更改库函数中 PySCF 库的代码，直接地对库中的函数与变量进行调试．

但是，Jupyter Notebook 在默认情况下，当内核准备完毕后，对库的 `.py` 文件的更改不会反映在程序的更改上；除非重启内核．这应当是因为 Python 在进行 `import` 命令时，会预编译被引入的 `.py` 文件为 `.pyc` 文件，从而只读取二进制的 `.pyc` 文件即可以高效地执行程序；而避免从 `.py` 文件先读取未编译的代码，再经过 Python 解释器编译．

事实上，Jupyter Notebook 在加入下述 Magic Command 后，可以在同一个内核进程中，对库函数的更改作出响应．但想必因为要进行库函数的 `.py` 文件进行额外的编译操作，因此执行效率多少会慢一些．

In [ ]:
%load_ext autoreload
%autoreload 2

<div class="alert alert-warning">

**警告**

修改库程序始终是危险操作！在修改之前至少需要作一个备份！

</div>

<div class="alert alert-info">

**提示**

一般库函数的位置在 Anaconda 安装文件夹下 `lib\python3.*\site-packages\pyscf` 中．

</div>

## 顶层函数计算 RHF 与 RMP2

引入库函数时，只需要 PySCF 与 NumPy 库即可；但 PySCF 的库经常需要手动引入．这些库需要通过查看源代码或者文档来了解．

In [ ]:
from pyscf import gto, scf, mp, ao2mo
import numpy as np

构建分子可以通过下述命令进行：

In [ ]:
mol = gto.Mole()
mol.atom = """
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
"""
mol.basis = "6-31G"
mol.build()

RHF 能量计算可以通过下述两行命令给出：

In [ ]:
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-13
energy_RHF = scf_eng.kernel()

RMP2 能量计算也可以通过两行命令给出：

In [ ]:
mp2_eng = mp.MP2(scf_eng)
energy_RMP2_corr, _ = mp2_eng.kernel()

这些结果应当能与 Gaussian 进行比对，误差应当在第七位小数左右．Gaussian 的输入卡可以是：

```
#p MP2(Full)/6-31G

H2O

0 1
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
```

## HF 中间矩阵与变量

在 Post-HF 计算与程序编写过程中，我们会经常使用到 HF 中间矩阵．在这里我们作简单的介绍．

### 系数矩阵 $C_{\mu p}$

In [ ]:
C = scf_eng.mo_coeff

### 本征向量 $\varepsilon_p$

In [ ]:
e = scf_eng.mo_energy

### 轨道数与电子数

以下代码依次定义原子轨道数 $n_\mathrm{AO}$、分子轨道数 $n_\mathrm{MO}$、电子数 $n_\mathrm{elec}$、占据轨道数 $n_\mathrm{occ}$、未占轨道数 $n_\mathrm{vir}$．

In [ ]:
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

由于我们已经假设了闭壳层以及没有轨道冻结，同时没有强基组线性依赖，因此以下的关系成立：

* $n_\mathrm{AO} = n_\mathrm{MO}$

In [ ]:
nao == nmo

* $n_\mathrm{elec} = 2 n_\mathrm{occ}$

In [ ]:
nelec == 2 * nocc

### AO 基组密度 $D_{\mu \nu}$

AO 基组密度可以通过下述公式获得：

\begin{equation}
D_{\mu \nu} = C_{\mu i} D_{ij} C_{\nu j}
\end{equation}

其中，$D_{ij} = 2 \delta_{ij}$ 是 MO 基组密度矩阵．注意到这里使用了 RHF 条件，即 $\alpha$ 与 $\beta$ 轨道密度相等，因此这里所使用的密度可能与不少程序的应用或教科书的定义相差两倍．

在 PySCF 中，可以使用 `scf.make_rdm1` 导出 AO 基组密度．我们可以验证这两种方法所导出的密度是相等的．

In [ ]:
D = scf_eng.make_rdm1()
np.allclose(C[:, :nocc] @ (2 * np.eye(nocc)) @ C[:, :nocc].T, D)

### AO 基组 Fock 矩阵 $F_{\mu \nu}$

这里我们仅仅写出 Fock 矩阵的导出方式．我们可以在对 AO 基组电子积分作简单介绍后，验证下述 Fock 矩阵的合理性．现在我们可以验证的，是 AO 基组 Fock 矩阵与轨道能之间的对应关系：

\begin{equation}
C_{\mu p} F_{\mu \nu} C_{\nu q} = \delta_{pq} \varepsilon_{p}
\end{equation}

In [ ]:
F = scf_eng.get_fock()
np.allclose(C.T @ F @ C, np.eye(nmo) * e)

<div class="alert alert-info">

**提示**

上述的验证在默认的 SCF 阈值 $10^{-10}$ 下，可能会给出 False 的判断．这是为何 [执行 SCF 计算](#顶层函数计算-RHF-与-RMP2) 时需要额外指定 SCF 收敛阈值的原因．

</div>

## 电子积分与导出矩阵

### AO 基组 Coulomb 矩阵 $J_{\mu \nu} [\mathbf{D}]$